In [ ]:
import requests
import math
import pandas as pd
API_KEY = open('../.env').read().strip()
PAYSAGE_URL = 'https://api.paysage.dataesr.ovh/structures'

In [2]:
def get_all_ids(paysage_id):
    URL = f'{PAYSAGE_URL}/{paysage_id}/identifiers?limit=500'
    r = requests.get(URL, headers={'X-API-KEY': API_KEY}).json()
    res = r['data']
    for e in res:
        for f in ['createdBy', 'createdAt', 'id', 'resourceId']:
            if f in e:
                del e[f]
    res.append({'type': 'paysage', 'value': paysage_id, 'active': True})
    return res

In [3]:
def get_all_paysage_structures():
    skip = 0
    limit = 20000
    r_paysage = requests.get(f'{PAYSAGE_URL}?limit={limit}&skip={skip}', headers={'X-API-KEY': API_KEY}).json()
    data = r_paysage['data']
    total = r_paysage['totalCount']
    
    nb_page = math.ceil(total / limit)
    for p in range (1, nb_page+1):
        print(p, end=',')
        skip = p * limit
        r_paysage = requests.get(f'{PAYSAGE_URL}?limit={limit}&skip={skip}', headers={'X-API-KEY': API_KEY}).json()
        data += r_paysage['data']
    assert(len(data) == total)
    print(f'{total} structures retrieved from paysage')
    return data

In [4]:
structures_paysage = get_all_paysage_structures()

1,12279 structures retrieved from paysage


In [5]:
active_structures_paysage = [k for k in structures_paysage if k.get('structureStatus') == 'active']
len(active_structures_paysage)

10895

In [6]:
for ix, e in enumerate(active_structures_paysage):
    if ix%500 == 0:
        print(ix, end=',')
    e['ids'] = get_all_ids(e['id'])

0,500,1000,1500,2000,2500,3000,3500,4000,4500,5000,5500,6000,6500,7000,7500,8000,8500,9000,9500,10000,10500,

In [7]:
id_from_paysage = []
for e in active_structures_paysage:
    has_id = False
    elt = {}
    if len(e['ids']) < 2:
        continue
    elt['name'] = e['currentName']['usualName']
    if 'city' in e['currentLocalisation']:
        elt['city'] = e['currentLocalisation']['city']
    if 'country' in e['currentLocalisation']:
        elt['country'] = e['currentLocalisation']['country']
    for x in e['ids']:
        if x['type'] in ['paysage', 'wikidata', 'siret', 'ror', 'grid', 'idref', 'rnsr', 'cnrs', 'ed']:
            id_type = x['type']
            if True or (x.get('active') != False):
                elt[id_type] = x['value']
                if id_type == 'siret':
                    elt['siren'] = x['value'][0:9]
                if id_type != 'paysage':
                    has_id = True
    if has_id:
        id_from_paysage.append(elt)

In [8]:
df = pd.DataFrame(id_from_paysage)

In [9]:
for f in ['siren', 'siret', 'idref', 'ed']:
    df[f] = df[f].astype(str)

In [10]:
df.to_json('paysage.jsonl', lines=True, orient='records')

In [ ]:
df[df.ror=='057qpr032']